In [355]:
import lasio
import pandas as pd


In [356]:
las = lasio.read(r"C:\Users\srorta\Desktop\gapapps\yellowtail-2st_gapapps.las") #change las file path


In [357]:
orig_las_df = las.df()
reset_index_df = orig_las_df.reset_index()
reset_index_df

,DEPTH,AI,AI_INSITU,BS,CALI,DRHO,DTC,DTS_FAST,DTS_SLOW,FPRESS,...,RHOF_FOIL,RHOF_FWAT,ROP,SALW,SWT,VCLAY,VP,VPVS,VS,VSH
0,1832.0004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2654.1482,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1832.1528,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2654.3728,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1832.3052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2654.5977,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1832.4576,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2654.8225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1832.6100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2655.0474,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24917,5629.3512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10453.7910,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24918,5629.5036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10454.0742,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24919,5629.6560,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10454.3574,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24920,5629.8084,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10454.6416,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [358]:
cds_df = pd.read_excel(r"C:\Users\srorta\Desktop\Dev_Surv\Yellowtail-2 ST.xlsx") #change deviation survey (from WIRE)
cds_df
well_name = cds_df["Well Name"][0]
well_name

'Yellowtail-2 ST'

In [359]:
tvddepthlist = []
mddepthlist =[]
latitude =[]
longitude=[]
easting = []
northing = []
for i,row in reset_index_df.iterrows():
    #find high and low indices inside the CDS table
    mddepthlist.append(round(row['DEPTH'],4))
    searchindex = 0
    while searchindex < len(cds_df) and cds_df["tvdrt"][searchindex] < row['DEPTH']:
        searchindex += 1
    lowindex = max(searchindex -1, 0)
    highindex = min(searchindex, len(cds_df)-1)
    print(lowindex)
    print(highindex)
    print(row['DEPTH'])
    
    #handle edge cases
    if lowindex == highindex:
        md = None
        tvd = None
        lat = None
        long = None
        east = None
        north = None
        if (lowindex ==0):
            tvd = cds_df["tvdrt"][0]
            md = cds_df["measuredDepth"][0]
            lat = cds_df["Latitude"][0]
            long = cds_df["Longitude"][0]
            east = cds_df["easting"][0]
            north = cds_df["northing"][0]
        else:
            searchindex = len(cds_df)-1
            while cds_df["tvdrt"][searchindex] is None and searchindex >0:
                searchindex -= 1
            tvd = cds_df["tvdrt"][searchindex]
            md = cds_df['measuredDepth'][searchindex]
            lat = cds_df['Latitude'][searchindex]
            long = cds_df['Longitude'][searchindex]
            east = cds_df["easting"][searchindex]
            north = cds_df["northing"][searchindex]
        delta = tvd-md
        tvdshiftdepth = row['DEPTH'] + delta
        tvdshiftdepth = round(tvdshiftdepth, 4)
        latitude.append(lat)
        longitude.append(long)
        easting.append(east)
        northing.append(north)
        tvddepthlist.append(tvdshiftdepth)
        print(tvdshiftdepth)
    # handle interpolation 
    if lowindex != highindex:
        lowtvddepth = cds_df["tvdrt"][lowindex]
        hightvddepth = cds_df["tvdrt"][highindex]
        lowmddepth = cds_df["measuredDepth"][lowindex]
        highmddepth = cds_df["measuredDepth"][highindex]
        lowlat = cds_df["Latitude"][lowindex]
        highlat = cds_df["Latitude"][highindex]
        lowlong = cds_df["Longitude"][lowindex]
        highlong = cds_df["Longitude"][highindex]  
        loweast = cds_df["easting"][lowindex]
        higheast = cds_df["easting"][highindex]
        lownorth = cds_df["northing"][lowindex]
        highnorth = cds_df["northing"][highindex]
        #if any of the above values are none, abort mission
        #if any(val is None for val in [lowCoreDepth, highCoreDepth, lowLogDepth, highLogDepth]):

        deltatvddepth = hightvddepth - lowtvddepth
        deltamddepth = highmddepth - lowmddepth 
        deltalat = highlat - lowlat
        deltalong = highlong - lowlong
        deltaeast = higheast - loweast
        deltanorth = highnorth - lownorth
        factor = (row['DEPTH'] - lowmddepth)/ deltamddepth
        tvdshiftdepth = factor * deltatvddepth + lowtvddepth
        lat = factor * deltalat + lowlat
        long = factor * deltalong + lowlong
        east = factor * deltaeast + loweast
        north = factor * deltanorth + lownorth
        tvdshiftdepth =round(tvdshiftdepth, 4)
        tvddepthlist.append(tvdshiftdepth)
        latitude.append(lat)
        longitude.append(long)
        easting.append(east)
        northing.append(north)
        print(tvdshiftdepth)


1
2
1832.0004
1832.0004
1
2
1832.1528
1832.1528
1
2
1832.3052
1832.3052
1
2
1832.4576
1832.4576
1
2
1832.61
1832.61
1
2
1832.7624
1832.7623
1
2
1832.9148
1832.9147
1
2
1833.0672
1833.0671
1
2
1833.2196
1833.2195
1
2
1833.372
1833.3719
1
2
1833.5244
1833.5243
1
2
1833.6768
1833.6767
1
2
1833.8292
1833.8291
1
2
1833.9816
1833.9815
1
2
1834.134
1834.1339
1
2
1834.2864
1834.2862
1
2
1834.4388
1834.4386
1
2
1834.5912
1834.591
1
2
1834.7436
1834.7434
1
2
1834.896
1834.8958
1
2
1835.0484
1835.0482
1
2
1835.2008
1835.2006
1
2
1835.3532
1835.353
1
2
1835.5056
1835.5054
1
2
1835.658
1835.6577
1
2
1835.8104
1835.8101
1
2
1835.9628
1835.9625
1
2
1836.1152
1836.1149
1
2
1836.2676
1836.2673
1
2
1836.42
1836.4197
1
2
1836.5724
1836.5721
1
2
1836.7248
1836.7245
1
2
1836.8772
1836.8769
1
2
1837.0296
1837.0292
1
2
1837.182
1837.1816
1
2
1837.3344
1837.334
1
2
1837.4868
1837.4864
1
2
1837.6392
1837.6388
1
2
1837.7916
1837.7912
1
2
1837.944
1837.9436
1
2
1838.0964
1838.096
1
2
1838.2488
1838.2484
1
2
1838

1907.4384
1907.4331
1
2
1907.5908
1907.5855
1
2
1907.7432
1907.7379
1
2
1907.8956
1907.8903
1
2
1908.048
1908.0427
1
2
1908.2004
1908.1951
1
2
1908.3528
1908.3475
1
2
1908.5052
1908.4999
1
2
1908.6576
1908.6523
1
2
1908.81
1908.8046
1
2
1908.9624
1908.957
1
2
1909.1148
1909.1094
1
2
1909.2672
1909.2618
1
2
1909.4196
1909.4142
1
2
1909.572
1909.5666
1
2
1909.7244
1909.719
1
2
1909.8768
1909.8714
1
2
1910.0292
1910.0238
1
2
1910.1816
1910.1761
1
2
1910.334
1910.3285
1
2
1910.4864
1910.4809
1
2
1910.6388
1910.6333
1
2
1910.7912
1910.7857
1
2
1910.9436
1910.9381
1
2
1911.096
1911.0905
1
2
1911.2484
1911.2429
1
2
1911.4008
1911.3953
1
2
1911.5532
1911.5476
1
2
1911.7056
1911.7
1
2
1911.858
1911.8524
1
2
1912.0104
1912.0048
1
2
1912.1628
1912.1572
1
2
1912.3152
1912.3096
1
2
1912.4676
1912.462
1
2
1912.62
1912.6144
1
2
1912.7724
1912.7668
1
2
1912.9248
1912.9192
1
2
1913.0772
1913.0715
1
2
1913.2296
1913.2239
1
2
1913.382
1913.3763
1
2
1913.5344
1913.5287
1
2
1913.6868
1913.6811
1
2
1913.839

5
1983.6384
1983.6185
4
5
1983.7908
1983.7708
4
5
1983.9432
1983.9232
4
5
1984.0956
1984.0756
4
5
1984.248
1984.228
4
5
1984.4004
1984.3803
4
5
1984.5528
1984.5327
4
5
1984.7052
1984.6851
4
5
1984.8576
1984.8375
4
5
1985.01
1984.9898
4
5
1985.1624
1985.1422
4
5
1985.3148
1985.2946
4
5
1985.4672
1985.447
4
5
1985.6196
1985.5993
4
5
1985.772
1985.7517
4
5
1985.9244
1985.9041
4
5
1986.0768
1986.0565
4
5
1986.2292
1986.2088
4
5
1986.3816
1986.3612
4
5
1986.534
1986.5136
4
5
1986.6864
1986.666
4
5
1986.8388
1986.8184
4
5
1986.9912
1986.9707
4
5
1987.1436
1987.1231
4
5
1987.296
1987.2755
4
5
1987.4484
1987.4279
4
5
1987.6008
1987.5802
4
5
1987.7532
1987.7326
4
5
1987.9056
1987.885
4
5
1988.058
1988.0374
4
5
1988.2104
1988.1897
4
5
1988.3628
1988.3421
4
5
1988.5152
1988.4945
4
5
1988.6676
1988.6469
4
5
1988.82
1988.7992
4
5
1988.9724
1988.9516
4
5
1989.1248
1989.104
4
5
1989.2772
1989.2564
4
5
1989.4296
1989.4087
4
5
1989.582
1989.5611
4
5
1989.7344
1989.7135
4
5
1989.8868
1989.8659
4
5
1990.

5
6
2040.7884
2040.7612
5
6
2040.9408
2040.9135
5
6
2041.0932
2041.0659
5
6
2041.2456
2041.2183
5
6
2041.398
2041.3707
5
6
2041.5504
2041.5231
5
6
2041.7028
2041.6755
5
6
2041.8552
2041.8279
5
6
2042.0076
2041.9802
5
6
2042.16
2042.1326
5
6
2042.3124
2042.285
5
6
2042.4648
2042.4374
5
6
2042.6172
2042.5898
5
6
2042.7696
2042.7422
5
6
2042.922
2042.8945
5
6
2043.0744
2043.0469
5
6
2043.2268
2043.1993
5
6
2043.3792
2043.3517
5
6
2043.5316
2043.5041
5
6
2043.684
2043.6565
5
6
2043.8364
2043.8088
5
6
2043.9888
2043.9612
5
6
2044.1412
2044.1136
5
6
2044.2936
2044.266
5
6
2044.446
2044.4184
5
6
2044.5984
2044.5708
5
6
2044.7508
2044.7232
5
6
2044.9032
2044.8755
5
6
2045.0556
2045.0279
5
6
2045.208
2045.1803
5
6
2045.3604
2045.3327
5
6
2045.5128
2045.4851
5
6
2045.6652
2045.6375
5
6
2045.8176
2045.7898
5
6
2045.97
2045.9422
5
6
2046.1224
2046.0946
5
6
2046.2748
2046.247
5
6
2046.4272
2046.3994
5
6
2046.5796
2046.5518
5
6
2046.732
2046.7042
5
6
2046.8844
2046.8565
5
6
2047.0368
2047.0089
5
6
2

2116.8052
7
8
2116.9884
2116.9576
7
8
2117.1408
2117.11
7
8
2117.2932
2117.2624
7
8
2117.4456
2117.4148
7
8
2117.598
2117.5672
7
8
2117.7504
2117.7196
7
8
2117.9028
2117.872
7
8
2118.0552
2118.0244
7
8
2118.2076
2118.1768
7
8
2118.36
2118.3291
7
8
2118.5124
2118.4815
7
8
2118.6648
2118.6339
7
8
2118.8172
2118.7863
7
8
2118.9696
2118.9387
7
8
2119.122
2119.0911
7
8
2119.2744
2119.2435
7
8
2119.4268
2119.3959
7
8
2119.5792
2119.5483
7
8
2119.7316
2119.7007
7
8
2119.884
2119.8531
7
8
2120.0364
2120.0055
7
8
2120.1888
2120.1579
7
8
2120.3412
2120.3103
7
8
2120.4936
2120.4627
7
8
2120.646
2120.6151
7
8
2120.7984
2120.7675
7
8
2120.9508
2120.9199
7
8
2121.1032
2121.0723
7
8
2121.2556
2121.2247
7
8
2121.408
2121.3771
7
8
2121.5604
2121.5295
7
8
2121.7128
2121.6819
7
8
2121.8652
2121.8343
7
8
2122.0176
2121.9867
7
8
2122.17
2122.139
7
8
2122.3224
2122.2914
7
8
2122.4748
2122.4438
7
8
2122.6272
2122.5962
7
8
2122.7796
2122.7486
7
8
2122.932
2122.901
7
8
2123.0844
2123.0534
7
8
2123.2368
2123.20

...

87
88
5315.712
5229.5731
87
88
5315.8644
5229.7232
87
88
5316.0168
5229.8733
87
88
5316.1692
5230.0234
87
88
5316.3216
5230.1735
87
88
5316.474
5230.3236
87
88
5316.6264
5230.4737
87
88
5316.7788
5230.6238
87
88
5316.9312
5230.7739
87
88
5317.0836
5230.924
87
88
5317.236
5231.0741
87
88
5317.3884
5231.2242
87
88
5317.5408
5231.3743
87
88
5317.6932
5231.5244
87
88
5317.8456
5231.6745
87
88
5317.998
5231.8246
87
88
5318.1504
5231.9747
87
88
5318.3028
5232.1248
87
88
5318.4552
5232.2749
87
88
5318.6076
5232.425
87
88
5318.76
5232.5751
87
88
5318.9124
5232.7252
87
88
5319.0648
5232.8753
87
88
5319.2172
5233.0254
87
88
5319.3696
5233.1755
87
88
5319.522
5233.3256
87
88
5319.6744
5233.4757
87
88
5319.8268
5233.6258
87
88
5319.9792
5233.776
87
88
5320.1316
5233.9261
87
88
5320.284
5234.0762
87
88
5320.4364
5234.2263
87
88
5320.5888
5234.3764
87
88
5320.7412
5234.5265
87
88
5320.8936
5234.6766
87
88
5321.046
5234.8267
87
88
5321.1984
5234.9768
87
88
5321.3508
5235.1269
87
88
5321.5032
5235.277

5304.4774
88
89
5391.912
5304.6275
88
89
5392.0644
5304.7776
89
90
5392.2168
5304.9242
89
90
5392.3692
5305.0743
89
90
5392.5216
5305.2244
89
90
5392.674
5305.3745
89
90
5392.8264
5305.5246
89
90
5392.9788
5305.6747
89
90
5393.1312
5305.8248
89
90
5393.2836
5305.9749
89
90
5393.436
5306.125
89
90
5393.5884
5306.2751
89
90
5393.7408
5306.4252
89
90
5393.8932
5306.5753
89
90
5394.0456
5306.7254
89
90
5394.198
5306.8755
89
90
5394.3504
5307.0256
89
90
5394.5028
5307.1757
89
90
5394.6552
5307.3258
89
90
5394.8076
5307.4759
89
90
5394.96
5307.626
89
90
5395.1124
5307.7761
89
90
5395.2648
5307.9262
89
90
5395.4172
5308.0763
89
90
5395.5696
5308.2264
89
90
5395.722
5308.3765
89
90
5395.8744
5308.5266
89
90
5396.0268
5308.6767
89
90
5396.1792
5308.8268
89
90
5396.3316
5308.9769
89
90
5396.484
5309.127
89
90
5396.6364
5309.2771
89
90
5396.7888
5309.4272
89
90
5396.9412
5309.5773
89
90
5397.0936
5309.7274
89
90
5397.246
5309.8775
89
90
5397.3984
5310.0276
89
90
5397.5508
5310.1777
89
90
5397.703

5448.9096
5360.7543
90
91
5449.062
5360.9044
90
91
5449.2144
5361.0545
90
91
5449.3668
5361.2047
90
91
5449.5192
5361.3548
90
91
5449.6716
5361.5049
90
91
5449.824
5361.655
90
91
5449.9764
5361.8051
90
91
5450.1288
5361.9552
90
91
5450.2812
5362.1054
90
91
5450.4336
5362.2555
90
91
5450.586
5362.4056
90
91
5450.7384
5362.5557
90
91
5450.8908
5362.7058
90
91
5451.0432
5362.8559
90
91
5451.1956
5363.0061
90
91
5451.348
5363.1562
90
91
5451.5004
5363.3063
90
91
5451.6528
5363.4564
90
91
5451.8052
5363.6065
90
91
5451.9576
5363.7567
90
91
5452.11
5363.9068
90
91
5452.2624
5364.0569
90
91
5452.4148
5364.207
90
91
5452.5672
5364.3571
90
91
5452.7196
5364.5072
90
91
5452.872
5364.6574
90
91
5453.0244
5364.8075
90
91
5453.1768
5364.9576
90
91
5453.3292
5365.1077
90
91
5453.4816
5365.2578
90
91
5453.634
5365.4079
90
91
5453.7864
5365.5581
90
91
5453.9388
5365.7082
90
91
5454.0912
5365.8583
90
91
5454.2436
5366.0084
90
91
5454.396
5366.1585
90
91
5454.5484
5366.3087
90
91
5454.7008
5366.4588
90


93
5506.0596
5415.9696
93
93
5506.212
5416.122
93
93
5506.3644
5416.2744
93
93
5506.5168
5416.4268
93
93
5506.6692
5416.5792
93
93
5506.8216
5416.7316
93
93
5506.974
5416.884
93
93
5507.1264
5417.0364
93
93
5507.2788
5417.1888
93
93
5507.4312
5417.3412
93
93
5507.5836
5417.4936
93
93
5507.736
5417.646
93
93
5507.8884
5417.7984
93
93
5508.0408
5417.9508
93
93
5508.1932
5418.1032
93
93
5508.3456
5418.2556
93
93
5508.498
5418.408
93
93
5508.6504
5418.5604
93
93
5508.8028
5418.7128
93
93
5508.9552
5418.8652
93
93
5509.1076
5419.0176
93
93
5509.26
5419.17
93
93
5509.4124
5419.3224
93
93
5509.5648
5419.4748
93
93
5509.7172
5419.6272
93
93
5509.8696
5419.7796
93
93
5510.022
5419.932
93
93
5510.1744
5420.0844
93
93
5510.3268
5420.2368
93
93
5510.4792
5420.3892
93
93
5510.6316
5420.5416
93
93
5510.784
5420.694
93
93
5510.9364
5420.8464
93
93
5511.0888
5420.9988
93
93
5511.2412
5421.1512
93
93
5511.3936
5421.3036
93
93
5511.546
5421.456
93
93
5511.6984
5421.6084
93
93
5511.8508
5421.7608
93
93
5

93
93
5563.2096
5473.1196
93
93
5563.362
5473.272
93
93
5563.5144
5473.4244
93
93
5563.6668
5473.5768
93
93
5563.8192
5473.7292
93
93
5563.9716
5473.8816
93
93
5564.124
5474.034
93
93
5564.2764
5474.1864
93
93
5564.4288
5474.3388
93
93
5564.5812
5474.4912
93
93
5564.7336
5474.6436
93
93
5564.886
5474.796
93
93
5565.0384
5474.9484
93
93
5565.1908
5475.1008
93
93
5565.3432
5475.2532
93
93
5565.4956
5475.4056
93
93
5565.648
5475.558
93
93
5565.8004
5475.7104
93
93
5565.9528
5475.8628
93
93
5566.1052
5476.0152
93
93
5566.2576
5476.1676
93
93
5566.41
5476.32
93
93
5566.5624
5476.4724
93
93
5566.7148
5476.6248
93
93
5566.8672
5476.7772
93
93
5567.0196
5476.9296
93
93
5567.172
5477.082
93
93
5567.3244
5477.2344
93
93
5567.4768
5477.3868
93
93
5567.6292
5477.5392
93
93
5567.7816
5477.6916
93
93
5567.934
5477.844
93
93
5568.0864
5477.9964
93
93
5568.2388
5478.1488
93
93
5568.3912
5478.3012
93
93
5568.5436
5478.4536
93
93
5568.696
5478.606
93
93
5568.8484
5478.7584
93
93
5569.0008
5478.9108
93
9

In [360]:
depth_view_data = {"MD (m)": mddepthlist, "Tvd (m)": tvddepthlist, "Latitude (DD)WGS84": latitude, "Longitude (DD) WGS84": longitude, "Easting (m) WGS84 UTM Zone 21N": easting, "Northing (m) WGS84 UTM Zone 21N": northing}
depth_view_df = pd.DataFrame(depth_view_data)
depth_view_df

,MD (m),Tvd (m),Latitude (DD)WGS84,Longitude (DD) WGS84,Easting (m) WGS84 UTM Zone 21N,Northing (m) WGS84 UTM Zone 21N
0,1832.0004,1832.0004,7.947050,-56.684761,534743.830004,878457.310000
1,1832.1528,1832.1528,7.947050,-56.684761,534743.831510,878457.310004
2,1832.3052,1832.3052,7.947050,-56.684761,534743.833017,878457.310007
3,1832.4576,1832.4576,7.947050,-56.684761,534743.834523,878457.310011
4,1832.6100,1832.6100,7.947050,-56.684761,534743.836029,878457.310014
...,...,...,...,...,...,...
24917,5629.3512,5539.2612,7.942506,-56.684667,534754.346000,877955.030000
24918,5629.5036,5539.4136,7.942506,-56.684667,534754.346000,877955.030000
24919,5629.6560,5539.5660,7.942506,-56.684667,534754.346000,877955.030000
24920,5629.8084,5539.7184,7.942506,-56.684667,534754.346000,877955.030000


In [361]:
depth_view_df["MD-TVD Delta (m)"] = depth_view_df["MD (m)"]-depth_view_df["Tvd (m)"]
depth_view_df["Reference Elevation (m)"] = 32 #change value based on well
depth_view_df["Water Depth (m)"] = 1800 #change value based on well
depth_view_df["Depth bml (m)"] = depth_view_df["Tvd (m)"]- (depth_view_df["Water Depth (m)"]+depth_view_df["Reference Elevation (m)"])
depth_view_df["VS (m/s)"] = reset_index_df["VS"] #for wells with VS_XW change from "VS" to "VS_XW"
depth_view_df["Well Name"] = well_name
depth_view_df

,MD (m),Tvd (m),Latitude (DD)WGS84,Longitude (DD) WGS84,Easting (m) WGS84 UTM Zone 21N,Northing (m) WGS84 UTM Zone 21N,MD-TVD Delta (m),Reference Elevation (m),Water Depth (m),Depth bml (m),VS (m/s),Well Name
0,1832.0004,1832.0004,7.947050,-56.684761,534743.830004,878457.310000,0.00,32,1800,0.0004,NaN,Yellowtail-2 ST
1,1832.1528,1832.1528,7.947050,-56.684761,534743.831510,878457.310004,0.00,32,1800,0.1528,NaN,Yellowtail-2 ST
2,1832.3052,1832.3052,7.947050,-56.684761,534743.833017,878457.310007,0.00,32,1800,0.3052,NaN,Yellowtail-2 ST
3,1832.4576,1832.4576,7.947050,-56.684761,534743.834523,878457.310011,0.00,32,1800,0.4576,NaN,Yellowtail-2 ST
4,1832.6100,1832.6100,7.947050,-56.684761,534743.836029,878457.310014,0.00,32,1800,0.6100,NaN,Yellowtail-2 ST
...,...,...,...,...,...,...,...,...,...,...,...,...
24917,5629.3512,5539.2612,7.942506,-56.684667,534754.346000,877955.030000,90.09,32,1800,3707.2612,NaN,Yellowtail-2 ST
24918,5629.5036,5539.4136,7.942506,-56.684667,534754.346000,877955.030000,90.09,32,1800,3707.4136,NaN,Yellowtail-2 ST
24919,5629.6560,5539.5660,7.942506,-56.684667,534754.346000,877955.030000,90.09,32,1800,3707.5660,NaN,Yellowtail-2 ST
24920,5629.8084,5539.7184,7.942506,-56.684667,534754.346000,877955.030000,90.09,32,1800,3707.7184,NaN,Yellowtail-2 ST


In [362]:
predicted_syn = "Predicted"
measured_syn = "Measured"
depth_view_df.to_excel(f"c:\\users\\srorta\\Desktop\\tvd_shear_final\\{well_name}_{measured_syn}.xlsx") #change predicted and measured based on log curve